## Expert Knowledge Worker

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports libary

import os
import glob
from dotenv import load_dotenv
import gradio as gr

d:\Code\Job-Aware-RAG-Agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# imports for langchain

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [10]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [12]:
folders = glob.glob("knowledge_base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [13]:
len(documents)

87

In [14]:
documents[24]

Document(metadata={'source': 'knowledge_base\\Deep_Learning\\fundamentals\\weight_initialization.md', 'doc_type': 'Deep_Learning'}, page_content='---\ntitle: Weight Initialization\ndescription: Giải thích các phương pháp khởi tạo trọng số trong neural network, công thức toán, tác động tới huấn luyện và cách áp dụng.\ntags: [deep-learning, fundamentals, neural-network, weight-initialization]\n---\n\n# Weight Initialization\n\n## 1. Tóm tắt khái niệm (Definition)\nWeight initialization là quá trình **gán giá trị ban đầu cho trọng số của neural network** trước khi huấn luyện.  \nMục tiêu: **tránh gradient vanish/explode, giúp hội tụ nhanh hơn**.\n\n---\n\n## 2. Mục đích & khi nào dùng (Use Cases)\n- Tránh **vanishing/exploding gradient**.  \n- Tăng tốc hội tụ khi huấn luyện.  \n- Giúp mạng học biểu diễn phi tuyến tốt hơn.  \n- Dùng trước bước training, cho mọi loại layer dense, convolutional, RNN.\n\n---\n\n## 3. Các phương pháp phổ biến\n\n### 3.1 Zero Initialization\n```\n\nW = 0\n\n```

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [16]:
len(chunks)

432

In [17]:
chunks[6]

Document(metadata={'source': 'knowledge_base\\Deep_Learning\\architectures\\cnn_architectures.md', 'doc_type': 'Deep_Learning'}, page_content='---\n\n## 7. So sánh với khái niệm liên quan (Comparison)\n\n* Basic CNN vs ResNet vs VGG vs Inception:\n\n  * Basic CNN: layer nối tiếp, đơn giản.\n  * VGG: nhiều layer nhỏ 3x3, đơn giản nhưng sâu.\n  * ResNet: skip connections, giảm vanishing gradient.\n  * Inception: multi-scale filter, giảm tham số mà giữ hiệu suất.\n\n---\n\n## 8. Ứng dụng trong thực tế (Practical Insights)\n\n* VGG/ResNet/Inception dùng trong ImageNet competition.\n* ResNet thường dùng cho các dự án detection/segmentation.\n* Inception hoặc EfficientNet tối ưu tốc độ và accuracy.\n\n---\n\n## 9. Câu hỏi phỏng vấn (Interview Questions)\n\n* Các kiến trúc CNN khác nhau điểm gì?\n* Skip connection giúp gì trong ResNet?\n* BatchNorm hoạt động thế nào?\n* Khi nào chọn VGG thay vì ResNet?\n\n---\n\n## 10. TL;DR (Short Summary)')

In [18]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: Deep_Learning, OOP, Preprocessing, DSA, Machine_Learning, Python


In [21]:
for chunk in chunks:
    if 'Hội tụ' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='---

## 7. So sánh với khái niệm liên quan (Comparison)

| Optimizer      | Khi nào dùng          | Ưu điểm                | Nhược điểm                    |
| -------------- | --------------------- | ---------------------- | ----------------------------- |
| SGD            | Dữ liệu nhỏ, đơn giản | Dễ hiểu                | Chậm, oscillation             |
| SGD + Momentum | Thường xuyên          | Hội tụ nhanh hơn       | Thêm hyperparameter           |
| NAG            | Thường xuyên          | Dự đoán gradient trước | Thêm tính toán                |
| AdaGrad        | Sparse features       | LR tự điều chỉnh       | LR giảm quá nhanh             |
| RMSProp        | RNN / LSTM            | Giữ LR ổn định         | Hyperparameter cần tinh chỉnh |
| Adam           | Hầu hết NN hiện đại   | Nhanh, ổn định         | Hyperparameter cần tune       |

---

## 8. Ứng dụng trong thực tế (Practical Insights)' metadata={'source': 'knowledge_base\\Deep_Learning\\fundamentals\\optimi